Questions Formule Un 

In [1]:
# Librairies
import pandas as pd
import os
import csv
from datetime import datetime

In [2]:
# Chargement des tables utiles
constructors = pd.read_csv(os.path.join("donnees_formule_un", "constructors.csv"))
constructors.columns = constructors.columns.str.strip()

constructors_standings = pd.read_csv(os.path.join("donnees_formule_un", "constructor_standings.csv"))
constructors_standings.columns = constructors_standings.columns.str.strip()

laps = pd.read_csv(os.path.join("donnees_formule_un", "lap_times.csv"))
laps.columns = laps.columns.str.strip()

races = pd.read_csv(os.path.join("donnees_formule_un", "races.csv"))
races.columns = races.columns.str.strip()
races['name']= races['name'].str.replace('"', '').str.strip()

drivers = pd.read_csv(os.path.join("donnees_formule_un", "drivers.csv")\
    , on_bad_lines='skip')
drivers.columns = drivers.columns.str.strip()
drivers["forename"] = drivers["forename"].str.replace('"', '').str.strip()
drivers["surname"] = drivers["surname"].str.replace('"', '').str.strip()
drivers["nom_complet"] = drivers["forename"] + " " + drivers["surname"]

results = pd.read_csv(os.path.join("donnees_formule_un", "results.csv"))
results.columns = results.columns.str.strip()

pit = pd.read_csv(os.path.join("donnees_formule_un", "pit_stops.csv"))
pit.columns = pit.columns.str.strip()

circuits = pd.read_csv(os.path.join("donnees_formule_un", "races.csv"))
circuits.columns = circuits.columns.str.strip()

Question 1 : Quels sont les pilotes qui ont gagné au moins 30 courses ? 

In [3]:
def pilotes_30_victoires(results):
    """
    Retourne les identifiants des pilotes ayant remporté au moins 30 courses.

    Parameters
    ----------
    results : DataFrame
        DataFrame contenant les résultats des courses avec au moins une colonne 'driverId'
        et une colonne 'positionOrder' indiquant la position finale du pilote dans la course.

    Returns
    -------
    Index
        Liste des 'driverId' des pilotes ayant remporté au moins 30 courses.
    """
    victoires = results[results['positionOrder'] == 1].groupby('driverId').size()
    pilotes_30_victoires = victoires[victoires >= 30].index

    return pilotes_30_victoires


def nom_pilotes_30_victoires(results, drivers):
    """
    Retourne les noms des pilotes ayant remporté au moins 30 courses.
    Retourne également leur nombre de victoires.

    Parameters
    ----------
    results : DataFrame
        DataFrame contenant les résultats des courses avec au moins une colonne 'driverId'
        et une colonne 'positionOrder' indiquant la position finale du pilote dans la course.

    drivers : DataFrame
        DataFrame contenant les informations des pilotes, avec au moins les colonnes 'driverId',
        'forename' et 'surname'.

    Returns
    -------
    list of str :
        "Prénom Nom (nombre victoires victoires)"
        pour les pilotes ayant au moins 30 victoires.
    """
    pilotes_30_victoire = pilotes_30_victoires(results)

    # Filtrer le DataFrame des pilotes pour obtenir les noms
    pilotes_info = drivers[drivers['driverId'].isin(pilotes_30_victoire)]\
        [['forename','surname', 'driverId']]
    victoires = results[results['positionOrder'] == 1].groupby('driverId').size()
    pilotes_info['victoires'] = pilotes_info['driverId'].map(victoires)
    noms_pilotes_victoires = pilotes_info.apply(
        lambda row: f"{row['forename']} {row['surname']} ({row['victoires']} victoires)", axis=1
    ).tolist()

    return noms_pilotes_victoires

nom_30_victoires = nom_pilotes_30_victoires(results, drivers)

print("Les pilotes ayant au moins 30 victoires sont :")
for pilote in nom_30_victoires:
    print(f"- {pilote}")



Les pilotes ayant au moins 30 victoires sont :
- Lewis Hamilton (104 victoires)
- Fernando Alonso (32 victoires)
- Sebastian Vettel (53 victoires)
- Michael Schumacher (91 victoires)
- Nigel Mansell (31 victoires)
- Ayrton Senna (41 victoires)
- Alain Prost (51 victoires)
- Max Verstappen (61 victoires)


Question 1 en python pur :

Question 2 : Quel est le classement des pilotes à l'issue de la saison 2023 ? 

In [4]:
# Fusion des données
races_results = pd.merge(races, results, on='raceId')
races_results = pd.merge(races_results, drivers[['driverId', 'nom_complet']], on='driverId')

races_results_2023 = races_results[races_results['year'] == 2023]

# Calcul des points totaux
points_totals = races_results_2023.groupby('driverId')['points'].sum().reset_index()

# Comptage des positions (1ère, 2ème, etc.)
positions_counts = races_results_2023.groupby(['driverId', 'positionOrder']).size().unstack(fill_value=0)

# Fusion avec les noms
classement = pd.merge(points_totals, drivers[['driverId', 'nom_complet']], on='driverId')
classement = pd.merge(classement, positions_counts, on='driverId', how='left')

# Remplacer les colonnes vides par 0 si certaines positions ne sont pas occupées
for pos in range(1, 21):
    if pos not in classement.columns:
        classement[pos] = 0

# Tri : d'abord par points, puis 1res places, 2des places, etc.
tri = ['points'] + list(range(1, 21))
classement = classement.sort_values(by=tri, ascending=[False] + [False]*20).reset_index(drop=True)

# Affichage final
colonnes_a_afficher = ['nom_complet', 'points'] + list(range(1, 11))  # on peut afficher que les 10 premières places
print(classement[colonnes_a_afficher])

         nom_complet  points   1  2  3  4  5  6  7  8  9  10
0     Max Verstappen   530.0  19  2  0  0  1  0  0  0  0   0
1       Sergio Pérez   260.0   2  4  3  5  1  2  0  1  0   1
2     Lewis Hamilton   217.0   0  3  3  3  3  4  1  2  1   0
3    Fernando Alonso   198.0   0  3  5  1  2  1  3  1  3   0
4    Charles Leclerc   185.0   0  3  3  4  1  1  3  0  1   0
5       Lando Norris   184.0   0  6  1  1  2  1  2  1  2   0
6       Carlos Sainz   178.0   1  0  2  2  5  5  0  2  0   1
7     George Russell   157.0   0  0  2  3  4  3  3  2  0   0
8      Oscar Piastri    82.0   0  1  1  1  1  1  1  2  1   2
9       Lance Stroll    68.0   0  0  0  1  2  2  2  0  3   2
10      Esteban Ocon    56.0   0  0  1  1  0  0  1  4  2   3
11      Pierre Gasly    54.0   0  0  1  0  0  2  2  1  2   3
12   Alexander Albon    25.0   0  0  0  0  0  0  2  2  2   1
13      Yuki Tsunoda    14.0   0  0  0  0  0  0  0  2  1   3
14   Valtteri Bottas    10.0   0  0  0  0  0  0  0  2  0   2
15   Nico Hülkenberg    

Question 3 : Quel est le podium des écurries à la fin de la saison en fonction des années ? 

In [5]:
# Fusion des tables
fusion = pd.merge(constructors_standings, races, on='raceId')
fusion = fusion[['raceId', 'constructorId', 'points', 'position', 'year']]
fusion_now = pd.merge(fusion, constructors, on='constructorId')

# Trouver le dernier raceId (la dernière course) pour chaque année
dernieres_courses = fusion_now.groupby('year')['raceId'].max().reset_index()

# Filtrer pour ne garder que les lignes correspondant à ces dernières courses
fusion_finale = pd.merge(fusion_now, dernieres_courses, on=['year', 'raceId'])

fusion_finale = fusion_finale[fusion_finale['position'].isin([1,2,3]) ]
fusion_finale = fusion_finale[['year', 'name', 'points', 'position']]
fusion_finale['name'] = fusion_finale['name'].str.replace('"', '').str.strip()
fusion_finale = fusion_finale.sort_values(by=['year', 'position'])


# Parcourir les années uniques en ordre décroissant
for year in sorted(fusion_finale['year'].unique(), reverse=True):
    podium = fusion_finale[fusion_finale['year'] == year]

    # Récupérer les positions du podium pour cette année
    num1 = podium[podium['position'] == 1].iloc[0]
    num2 = podium[podium['position'] == 2].iloc[0]
    num3 = podium[podium['position'] == 3].iloc[0]

    print(f"En {year}, le podium des écuries était : {num1['name']} "
          f"({num1['points']} points), suivi de {num2['name']} "
          f"({num2['points']} points) et enfin {num3['name']} "
          f"({num3['points']} points).")


En 2024, le podium des écuries était : Red Bull (373.0 points), suivi de Ferrari (302.0 points) et enfin McLaren (295.0 points).
En 2023, le podium des écuries était : Red Bull (860.0 points), suivi de Mercedes (409.0 points) et enfin Ferrari (406.0 points).
En 2022, le podium des écuries était : Red Bull (759.0 points), suivi de Ferrari (554.0 points) et enfin Mercedes (515.0 points).
En 2021, le podium des écuries était : Mercedes (613.5 points), suivi de Red Bull (585.5 points) et enfin Ferrari (323.5 points).
En 2020, le podium des écuries était : Mercedes (573.0 points), suivi de Red Bull (319.0 points) et enfin McLaren (202.0 points).
En 2019, le podium des écuries était : Mercedes (739.0 points), suivi de Ferrari (504.0 points) et enfin Red Bull (417.0 points).
En 2018, le podium des écuries était : Mercedes (655.0 points), suivi de Ferrari (571.0 points) et enfin Red Bull (419.0 points).
En 2017, le podium des écuries était : Mercedes (668.0 points), suivi de Ferrari (522.0 poi

Question 4 :  Quels sont les meilleurs temps d'un tour de circuit (best lap time) pour chaque circuit, en 2023 ?

In [7]:
# Fusion des tables
fusion = pd.merge(laps, races, on='raceId')
fusion.columns = fusion.columns.str.strip()
fusion['name'] = fusion["name"].str.replace('"', '').str.strip()
# Extraire les années des courses
annees_courses = fusion[['year', 'raceId', 'name']].drop_duplicates()
# Extraire le temps le plus rapide
meilleur_temps_tour_par_course =\
    fusion.groupby(['raceId', 'year'])['milliseconds'].min().reset_index()
# Fusionner à nouveau pour inclure la colonne 'name' (nom de la course)
meilleur_temps_tour_par_course =\
    pd.merge(meilleur_temps_tour_par_course, fusion[['raceId', 'name']], on='raceId',
             how='left').drop_duplicates()
# Filtrer sur l'année 2023
meilleur_temps_tour_par_course_2023 =\
    meilleur_temps_tour_par_course[meilleur_temps_tour_par_course['year'] == 2023]
# Afficher le temps le plus rapide
print("\nMeilleur temps d'un tour de circuit par course en 2023 :")
print(f"{meilleur_temps_tour_par_course_2023[['name', 'milliseconds']]}")



Meilleur temps d'un tour de circuit par course en 2023 :
                            name  milliseconds
538121        Bahrain Grand Prix         93996
539176  Saudi Arabian Grand Prix         91906
540118     Australian Grand Prix         80235
541113     Azerbaijan Grand Prix        103370
542074          Miami Grand Prix         89708
543212         Monaco Grand Prix         75650
544725        Spanish Grand Prix         76330
546037       Canadian Grand Prix         74481
547352       Austrian Grand Prix         67012
548705        British Grand Prix         90275
549675      Hungarian Grand Prix         80504
550927        Belgian Grand Prix        107305
551742          Dutch Grand Prix         73837
553083        Italian Grand Prix         85072
554038      Singapore Grand Prix         95867
555122       Japanese Grand Prix         94183
556002          Qatar Grand Prix         84319
557007  United States Grand Prix         98139
558021    Mexico City Grand Prix         81334
55

Question 4 en python pur : 

In [8]:
# Lecture et stockage des données
races_dict = {}
race_ids_2023 = set()

with open(os.path.join("donnees_formule_un", "races.csv"), 'r', encoding='utf-8') as races_file:
    reader = csv.reader(races_file)
    header_races = next(reader)
    for row in reader:
        try:
            raceId = row[0].strip()
            year = int(row[1].strip())
            if year == 2023:
                races_dict[raceId] = row
                race_ids_2023.add(raceId)
        except (IndexError, ValueError):
            continue

# Fusionner les temps et les courses pour l'année 2023
with open(os.path.join("donnees_formule_un", "lap_times.csv"), 'r', encoding='utf-8') as laps_file:
    reader = csv.reader(laps_file)
    header_laps = next(reader)

    with open("fusion_2023.csv", 'w', newline='', encoding='utf-8') as output_file:
        writer = csv.writer(output_file)
        writer.writerow(header_laps + header_races)
        for row in reader:
            try:
                raceId = row[0].strip()
                if raceId in race_ids_2023:
                    ligne_fusionnee = row + races_dict[raceId]
                    writer.writerow(ligne_fusionnee)
            except IndexError:
                continue

# Lire et afficher les meilleurs temps par courses
meilleurs_temps = {}
with open("fusion_2023.csv", 'r', encoding='utf-8') as fusion_file:
    reader = csv.reader(fusion_file)
    header = next(reader)
    for i, row in enumerate(reader):
        try:
            nom_course = row[10].strip()
            temps_ms = int(row[5].strip())
            if nom_course not in meilleurs_temps or temps_ms < meilleurs_temps[nom_course]:
                meilleurs_temps[nom_course] = temps_ms
        except (IndexError, ValueError):
            continue


print("Meilleurs temps par course en 2023 :\n")
for nom, temps in meilleurs_temps.items():
    print(f"{nom} : {temps} ms")

Meilleurs temps par course en 2023 :

"Bahrain Grand Prix" : 93996 ms
"Saudi Arabian Grand Prix" : 91906 ms
"Australian Grand Prix" : 80235 ms
"Azerbaijan Grand Prix" : 103370 ms
"Miami Grand Prix" : 89708 ms
"Monaco Grand Prix" : 75650 ms
"Spanish Grand Prix" : 76330 ms
"Canadian Grand Prix" : 74481 ms
"Austrian Grand Prix" : 67012 ms
"British Grand Prix" : 90275 ms
"Hungarian Grand Prix" : 80504 ms
"Belgian Grand Prix" : 107305 ms
"Dutch Grand Prix" : 73837 ms
"Italian Grand Prix" : 85072 ms
"Singapore Grand Prix" : 95867 ms
"Japanese Grand Prix" : 94183 ms
"Qatar Grand Prix" : 84319 ms
"United States Grand Prix" : 98139 ms
"Mexico City Grand Prix" : 81334 ms
"São Paulo Grand Prix" : 72486 ms
"Las Vegas Grand Prix" : 95490 ms
"Abu Dhabi Grand Prix" : 86993 ms


Question 5 : Quel est le nombre de meilleur temps (best lap time) par pilote, en 2023 ? 

In [9]:
# Fusion des tables
fusion= pd.merge(laps, races, on='raceId')
fusion.columns = fusion.columns.str.strip()
fusionnow = pd.merge(fusion, drivers[['driverId', 'nom_complet']], on='driverId')

# Extraire les meilleurs temps par course
meilleur_temps_tour_par_course = (
    fusionnow.loc[fusionnow.groupby(['raceId', 'year'])['milliseconds'].idxmin()]
)

# Filtrer sur l'année 2023
meilleur_temps_tour_par_course_2023 = meilleur_temps_tour_par_course[
    meilleur_temps_tour_par_course['year'] == 2023
]
# regroupe par pilote
meilleur_temps_par_pilote = (
    meilleur_temps_tour_par_course_2023.groupby('nom_complet')
    .size()
    .reset_index(name='Nombre de meilleurs temps')
    .sort_values(by='Nombre de meilleurs temps', ascending=False)
)

print("\nNombre de meilleurs temps par pilote en 2023 :")
for index, row in meilleur_temps_par_pilote.iterrows():
    print(f"{row['nom_complet']} réalise {row['Nombre de meilleurs temps']} meilleurs temps en 2023.")



Nombre de meilleurs temps par pilote en 2023 :
Max Verstappen réalise 9 meilleurs temps en 2023.
Lewis Hamilton réalise 4 meilleurs temps en 2023.
Oscar Piastri réalise 2 meilleurs temps en 2023.
Sergio Pérez réalise 2 meilleurs temps en 2023.
Fernando Alonso réalise 1 meilleurs temps en 2023.
George Russell réalise 1 meilleurs temps en 2023.
Guanyu Zhou réalise 1 meilleurs temps en 2023.
Lando Norris réalise 1 meilleurs temps en 2023.
Yuki Tsunoda réalise 1 meilleurs temps en 2023.


Question 6 : Quel pilote a remporté le plus de fois le circuit de Spa-Francorchamps depuis 1950 ? 

In [10]:
# Fusion des tables
fusion = pd.merge(results, races, on='raceId')
spa_gagnant = pd.merge(drivers, fusion, on='driverId')

# Filtrer la table
spa_gagnant = spa_gagnant[spa_gagnant['circuitId'] == 13] # Garder le circuit Spa-Francorchamps
spa_gagnant = spa_gagnant[spa_gagnant['positionOrder'] == 1] # Garder les gagnants

spa_gagnant = (
    spa_gagnant.groupby('nom_complet') # Regrouper par pilote
    .size()
    .reset_index(name="Nombre de victoires")  # Renommer la colonne de comptage
    .sort_values(by="Nombre de victoires", ascending=False)  # Trier par nombre de victoires décroissant
)

top2 = spa_gagnant.head(2)
num_1 = top2.iloc[0]
num_2 = top2.iloc[1]

print(f"Le pilote qui a le plus grand nombre de victoires sur le circuit de \
Spa-francorchamps est {num_1['nom_complet']} avec {num_1['Nombre de victoires']} \
victoires, suivi par {num_2['nom_complet']} \
avec {num_2['Nombre de victoires']} victoires.")

Le pilote qui a le plus grand nombre de victoires sur le circuit de Spa-francorchamps est Michael Schumacher avec 6 victoires, suivi par Ayrton Senna avec 5 victoires.


Question 7 : Quel pilote a eu le plus d'accident en fonction des années ? 

Question 8 : Quel est le temps moyen des pit stop en fonction des années ? 
(Sacahant qu'on ne dispose pas des données avant 2011)

In [9]:

# Fusion des tables
fusion = pd.merge(races, pit, on='raceId')
fusion.columns = fusion.columns.str.strip()

pit_2023 = fusion[fusion['year'] == 2023]
pit_2020 = fusion[fusion['year'] == 2020]
pit_2016 = fusion[fusion['year'] == 2016]
pit_2013 = fusion[fusion['year'] == 2013]


# Filtre pour retirer les temps abérrants, valeurs extrêmes
def filtrer_pit_stops(df):
    q1 = df['milliseconds'].quantile(0.01)
    q99 = df['milliseconds'].quantile(0.99)
    limite_superieure = min(q99, 30000)  # max 30 sec
    return df[(df['milliseconds'] >= q1) & (df['milliseconds'] <= limite_superieure)]


pit_2023_filtre = filtrer_pit_stops(pit_2023)
pit_2020_filtre = filtrer_pit_stops(pit_2020)
pit_2016_filtre = filtrer_pit_stops(pit_2016)
pit_2013_filtre = filtrer_pit_stops(pit_2013)

# Calcul des moyennes filtrées
moy_2023_filtre = pit_2023_filtre['milliseconds'].mean()
moy_2020_filtre = pit_2020_filtre['milliseconds'].mean()
moy_2016_filtre = pit_2016_filtre['milliseconds'].mean()
moy_2013_filtre = pit_2013_filtre['milliseconds'].mean()

print(f"Le temps moyen (filtré) des pit stops en 2023 est de {moy_2023_filtre:.0f} ms.")
print(f"Il était de {moy_2020_filtre:.0f} ms en 2020, \
de {moy_2016_filtre:.0f} ms en 2016 et de {moy_2013_filtre:.0f} ms en 2013.")


Le temps moyen (filtré) des pit stops en 2023 est de 23567 ms.
Il était de 24326 ms en 2020, de 23733 ms en 2016 et de 22988 ms en 2013.


Question 9 : Quels circuits ont été le plus de fois concouru, depuis 1950 ?

In [20]:
# Fusion des tables
fusion = pd.merge(races, circuits, on= 'raceId')
fusion["name_x"] = fusion["name_x"].str.replace('"', '').str.strip()

# On regroupe par circuit et on compte le nombre de fois où il a était concurru
nb_fois_circuit = (
    fusion.groupby('name_x')
    .size()
    .reset_index(name="Nombre de fois où le circuit a été concurru")
    .sort_values(by="Nombre de fois où le circuit a été concurru", ascending=False)
)

top3 = nb_fois_circuit.head(3)
num_1 = top3.iloc[0]
num_2 = top3.iloc[1]
num_3 = top3.iloc[2]

print(f"Les circuits ayant été le plus de fois courus depuis 1950 sont le \
{num_1['name_x']} et l'{num_2['name_x']} courus \
{num_1['Nombre de fois où le circuit a été concurru']} fois, \
suivi par le {num_3['name_x']} \
couru {num_3['Nombre de fois où le circuit a été concurru']} fois.")

print(nb_fois_circuit)

Les circuits ayant été le plus de fois courus depuis 1950 sont le British Grand Prix et l'Italian Grand Prix courus 75 fois, suivi par le Monaco Grand Prix couru 70 fois.
                           name_x  Nombre de fois où le circuit a été concurru
9              British Grand Prix                                           75
24             Italian Grand Prix                                           75
33              Monaco Grand Prix                                           70
7              Belgian Grand Prix                                           69
20              German Grand Prix                                           64
19              French Grand Prix                                           62
45             Spanish Grand Prix                                           54
11            Canadian Grand Prix                                           53
8            Brazilian Grand Prix                                           47
52       United States Grand Prix      

Question 9 en python pur 

In [12]:
# Chemin vers les fichiers
fichier_races = os.path.join("donnees_formule_un", "races.csv")
fichier_circuits = os.path.join("donnees_formule_un", "circuits.csv")

# On charge circuits.csv dans un dictionnaire : circuitId → nom du circuit
circuits = {}
with open(fichier_circuits, mode="r", encoding="utf-8") as f:
    reader = csv.reader(f)
    header = [col.strip() for col in next(reader)]
    circuitId_index = header.index("circuitId")
    name_index = header.index("name")

    for row in reader:
        circuit_id = row[circuitId_index].strip()
        circuit_name = row[name_index].strip()
        circuits[circuit_id] = circuit_name

# On compte combien de fois chaque circuitId apparaît dans races.csv
compteur_circuits = {}

with open(fichier_races, mode="r", encoding="utf-8") as f:
    reader = csv.reader(f)
    header = [col.strip() for col in next(reader)]
    circuit_index = header.index("circuitId")

    for row in reader:
        circuit_id = row[circuit_index].strip()  # assure que c'est bien une chaîne nettoyée
        if circuit_id in compteur_circuits:
            compteur_circuits[circuit_id] += 1
        else:
            compteur_circuits[circuit_id] = 1

# le circuit avec le plus grand nombre de courses
max_circuit_id = None
max_count = 0

for cid, count in compteur_circuits.items():
    if count > max_count:
        max_count = count
        max_circuit_id = cid



# Résultat final
nom_circuit = circuits.get(max_circuit_id, "Nom inconnu")
print(f"Le circuit le plus souvent courru est : {nom_circuit} ({max_count} fois)")


Le circuit le plus souvent courru est : "Autodromo Nazionale di Monza" (74 fois)


Question 10 : 

In [ ]:
# Charger les données
df_results = pd.read_csv(os.path.join("donnees_formule_un", "results.csv"))

df_races = pd.read_csv(os.path.join("donnees_formule_un", "races.csv"))

df_circuits = pd.read_csv(os.path.join("donnees_formule_un", "circuits.csv"), on_bad_lines='skip')

df_drivers = pd.read_csv(os.path.join("donnees_formule_un", "drivers.csv"), on_bad_lines='skip')

# Nettoyage des colonnes
df_results.columns = df_results.columns.str.strip()

df_races.columns = df_races.columns.str.strip()

df_circuits.columns = df_circuits.columns.str.strip()

df_drivers.columns = df_drivers.columns.str.strip()

# Ajouter deux colonnes pour séparer les données de "time"
df_results["time_with_plus"] = df_results["time"].where(
    df_results["time"].astype(str).str.contains(r"\+"), None
)
df_results["time_without_plus"] = df_results["time"].where(
    ~df_results["time"].astype(str).str.contains(r"\+"), None
)

df_results_new = df_results.loc[
    :, ["raceId", "driverId", "time_with_plus", "time_without_plus"]
]

# Nettoyer les espaces autour des valeurs dans les colonnes "time_with_plus" et "time_without_plus"
df_results_new["time_with_plus"] = df_results_new["time_with_plus"].str.strip()
df_results_new["time_without_plus"] = df_results_new["time_without_plus"].str.strip()

# Replace string '\N' with actual NaN
df_results_new["time_without_plus"].replace("\\N", pd.NA, inplace=True)

# Filter rows where either column has a value (not null)
st_anne = df_results_new[
    df_results_new["time_with_plus"].notna()
    | df_results_new["time_without_plus"].notna()
]

# Filtrer les valeurs non nulles avant d'utiliser min()
filtered_time_with_plus = st_anne["time_with_plus"].dropna()

# Trouver la valeur minimale
min_time_with_plus = min(filtered_time_with_plus.astype(str))

# Afficher la ligne complète associée à cette valeur minimale
min_time_row = st_anne[st_anne["time_with_plus"] == min_time_with_plus]

# Ajouter la valeur de "time_without_plus" du premier ayant le même "raceId"
# dans une nouvelle colonne "best_time"

race_id = min_time_row["raceId"].values[0]
best_time = st_anne.loc[
    (st_anne["raceId"] == race_id) & st_anne["time_without_plus"].notna(),
    "time_without_plus"
].values[0]

min_time_row["best_time"] = best_time

# Charger à nouveau si nécessaire (ou s'assurer qu'on a toutes les colonnes dans df_results)
df_results_full = pd.read_csv(os.path.join("donnees_formule_un", "results.csv"))
df_results_full.columns = df_results_full.columns.str.strip()

# Convertir raceId et positionOrder en entiers au cas où
df_results_full["raceId"] = df_results_full["raceId"].astype(int)
df_results_full["positionOrder"] = pd.to_numeric(df_results_full["positionOrder"], errors='coerce')

# Extraire le driverId du gagnant pour ce raceId
winner_driver_id = df_results_full[
    (df_results_full["raceId"] == race_id) &
    (df_results_full["positionOrder"] == 1)
]["driverId"].values[0]

# Ajouter cette valeur à la ligne existante
min_time_row["driverId_Winner"] = winner_driver_id

# Extraire le raceId de min_time_row
race_id = min_time_row["raceId"].values[0]

# Filtrer pour obtenir les informations de la course
race_info = df_races[df_races["raceId"] == race_id][["name", "date", "circuitId"]]

# Récupérer circuitId et nom du GP (on a déjà race_info)
circuit_id = race_info["circuitId"].values[0]
nom_GP = race_info["name"].values[0]
date_GP = race_info["date"].values[0]

# Nom du circuit
nom_de_circuit = df_circuits[df_circuits["circuitId"] == circuit_id]["name"].values[0]

# Récupérer noms des pilotes
driver_id_winner = min_time_row["driverId_Winner"].values[0]
driver_id_second = min_time_row["driverId"].values[0]

# Nom du pilote gagnant
pilote_1er = df_drivers[df_drivers["driverId"] == driver_id_winner]
nom_du_pilote_1er = pilote_1er["forename"].values[0] + " " + pilote_1er["surname"].values[0]

# Nom du deuxième pilote
pilote_2eme = df_drivers[df_drivers["driverId"] == driver_id_second]
nom_du_pilote_2eme = pilote_2eme["forename"].values[0] + " " + pilote_2eme["surname"].values[0]

# Temps
temps_premier = min_time_row["best_time"].values[0]
ecart = min_time_row["time_with_plus"].values[0]

# Créer la table résultat
resultat = pd.DataFrame([{
    "nom_GP": nom_GP,
    "nom_de_circuit": nom_de_circuit,
    "date": date_GP,
    "nom_du_pilote_1er": nom_du_pilote_1er,
    "nom_du_pilote_2eme": nom_du_pilote_2eme,
    "temps_premier": temps_premier,
    "ecart": ecart
}])

# Afficher le résultat
phrase = (
    f"Lors du Grand Prix {nom_GP} disputé le {date_GP} sur le circuit de {nom_de_circuit}, "
    f"le pilote {nom_du_pilote_1er} a remporté la course avec un temps de {temps_premier}, "
    f"devançant {nom_du_pilote_2eme} de {ecart} secondes."
)

print(phrase)



Question 11 : Quel est le nombre de victoires par nation, entre 1950 et 2024 ?

In [15]:
# Fusion des tables
fusion = pd.merge(drivers, results, on="driverId")
fusion['nationality'] = fusion["nationality"].str.replace('"', '').str.strip()

victoire = fusion[fusion['positionOrder'] == 1]     # Garder les victoires

victoire_nation = (
    victoire.groupby('nationality')   # Regrouper par nation
    .size()
    .reset_index(name="Nombre de victoires par nation")
    .sort_values(by="Nombre de victoires par nation", ascending=False)
)

print(victoire_nation)


      nationality  Nombre de victoires par nation
6         British                             312
12         German                             179
5       Brazilian                             101
11         French                              81
9           Dutch                              61
10        Finnish                              57
2      Australian                              43
13        Italian                              43
3        Austrian                              41
1       Argentine                              38
19        Spanish                              35
0        American                              33
7        Canadian                              17
16  New Zealander                              12
20        Swedish                              12
4         Belgian                              11
18  South African                              10
14        Mexican                               8
8       Colombian                               7


Question 12 : Quel est l'âge moyen des pilotes en fonction des années ? 

In [16]:
# Fusion des tables
fusion_1 = pd.merge(drivers, results, on='driverId')
fusion_2 = pd.merge(fusion_1, races, on='raceId')

drivers_unique = fusion_2.drop_duplicates(subset=["driverId"]) # Vérifier les doublons

# Filtrer sur l'année 2023
driver_unique_2023 = drivers_unique[drivers_unique['year'] == 2023].copy()
driver_unique_2023.columns = driver_unique_2023.columns.str.strip()

# Convertir la date de naissance en format datetime pour calculer l'âge
driver_unique_2023['dob'] = driver_unique_2023['dob'].str.strip().str.replace('"', '')
driver_unique_2023['dob'] = pd.to_datetime(driver_unique_2023['dob'], format='%Y-%m-%d', errors='coerce')
driver_unique_2023.loc[:, 'dob'] = pd.to_datetime(driver_unique_2023['dob'])
driver_unique_2023[driver_unique_2023['dob'].isna()]

aujourdhui = datetime.now()
driver_unique_2023.loc[:, 'age'] = driver_unique_2023['dob'].apply(
    lambda x: aujourdhui.year - x.year - ((aujourdhui.month, aujourdhui.day) < (x.month, x.day))
)

age_moyen = driver_unique_2023['age'].mean()   # Calcul de l'âge moyen
print(f"L'âge moyen des pilotes en 2023 est de {age_moyen:.2f} ans")


# Idem pour l'année 2000 :
driver_unique_2000 = drivers_unique[drivers_unique['year'] == 2000].copy()
driver_unique_2000.columns = driver_unique_2000.columns.str.strip()
driver_unique_2000['dob'] = driver_unique_2000['dob'].str.strip().str.replace('"', '')
driver_unique_2000['dob'] = pd.to_datetime(driver_unique_2000['dob'], format='%Y-%m-%d', errors='coerce')
driver_unique_2000.loc[:, 'dob'] = pd.to_datetime(driver_unique_2000['dob'])
driver_unique_2000[driver_unique_2000['dob'].isna()]
aujourdhui_2000 = datetime(2000, 1, 1)  # Date du 1er janvier 2000
driver_unique_2000.loc[:, 'age'] = driver_unique_2000['dob'].apply(
    lambda x: aujourdhui_2000.year - x.year - ((aujourdhui.month, aujourdhui.day) < (x.month, x.day))
)
age_moyen = driver_unique_2000['age'].mean()
print(f"L'âge moyen des pilotes en 2000 est de {age_moyen:.2f} ans")


# Idem pour l'année 1980 :
driver_unique_1980 = drivers_unique[drivers_unique['year'] == 1980].copy()
driver_unique_1980.columns = driver_unique_1980.columns.str.strip()
driver_unique_1980['dob'] = driver_unique_1980['dob'].str.strip().str.replace('"', '')
driver_unique_1980['dob'] = pd.to_datetime(driver_unique_1980['dob'], format='%Y-%m-%d', errors='coerce')
driver_unique_1980.loc[:, 'dob'] = pd.to_datetime(driver_unique_1980['dob'])
driver_unique_1980[driver_unique_1980['dob'].isna()]
aujourdhui_1980 = datetime(1980, 1, 1)  # Date du 1er janvier 1980
driver_unique_1980.loc[:, 'age'] = driver_unique_1980['dob'].apply(
    lambda x: aujourdhui_1980.year - x.year - ((aujourdhui.month, aujourdhui.day) < (x.month, x.day))
)
age_moyen = driver_unique_1980['age'].mean()
print(f"L'âge moyen des pilotes en 1980 est de {age_moyen:.2f} ans")

# Idem pour l'année 1950 :
driver_unique_1950 = drivers_unique[drivers_unique['year'] == 1950].copy()
driver_unique_1950.columns = driver_unique_1950.columns.str.strip()
driver_unique_1950['dob'] = driver_unique_1950['dob'].str.strip().str.replace('"', '')
driver_unique_1950['dob'] = pd.to_datetime(driver_unique_1950['dob'], format='%Y-%m-%d', errors='coerce')
driver_unique_1950.loc[:, 'dob'] = pd.to_datetime(driver_unique_1950['dob'])
driver_unique_1950[driver_unique_1950['dob'].isna()]
aujourdhui_1950 = datetime(1950, 1, 1)  # Date du 1er janvier 1950
driver_unique_1950.loc[:, 'age'] = driver_unique_1950['dob'].apply(
    lambda x: aujourdhui_1950.year - x.year - ((aujourdhui.month, aujourdhui.day) < (x.month, x.day))
)
age_moyen = driver_unique_1950['age'].mean()
print(f"L'âge moyen des pilotes en 1950 est de {age_moyen:.2f} ans")

L'âge moyen des pilotes en 2023 est de 23.67 ans
L'âge moyen des pilotes en 2000 est de 32.00 ans
L'âge moyen des pilotes en 1980 est de 32.00 ans
L'âge moyen des pilotes en 1950 est de 37.68 ans
